In [1]:
import os
import dotenv

dotenv.load_dotenv()
from smolagents import CodeAgent, tool, VisitWebpageTool, DuckDuckGoSearchTool, AzureOpenAIServerModel
import math
from typing import Optional, Tuple

MODEL = AzureOpenAIServerModel(
    model_id=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_VERSION")
)

MODEL_CLEVER = AzureOpenAIServerModel(
    model_id="gpt-4o-2024-11-20",
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_VERSION")
)

/private/var/user/src/edu/hf_agents/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


In [ ]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

agent = CodeAgent(
    model=MODEL,
    tools=[DuckDuckGoSearchTool(max_results=5), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

result = agent.run(task)
result

In [ ]:
agent.planning_interval = 5

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")
detailed_report

## Multi-agent

In [3]:
from smolagents.utils import encode_image_base64, make_image_url
from PIL import Image

def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = MODEL_CLEVER
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True

web_agent = CodeAgent(
    model=MODEL,
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

manager_agent = CodeAgent(
    model=MODEL_CLEVER,
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
        "PIL"
    ],
    name="manager_agent",
    description="Steers and owns the process of finding information",
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

manager_agent.visualize()

CodeAgent | gpt-4o-2024-11-20
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy', 'PIL']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | gpt-4o-mini-2024-07-18
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a duckduckgo web search  │ query (`string`): The search      │
            │                             │ based on your query (think a      │ query to perform.                 │
            │                             │ Google search) then returns the   │                                   │
            │                             │ top search results.               │                                   │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.                         │ destination_coords (`array`):     │
            │                             │                       

In [4]:
# Using shorter prompt, let's see if the model can draw the plot without any hints
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!
""")

# manager_agent.run("""
# Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
# Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
# Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

# Here's an example of how to plot and return a map:
# import plotly.express as px
# df = px.data.carshare()
# fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
#      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
# fig.show()
# fig.write_image("saved_image.png")
# final_answer(fig)

# Never try to process strings using code: when you have a string to read, just print it and you'll see it.
# """)

╭──────────────────────────────────────────── New run - manager_agent ────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
╰─ AzureOpenAIServerModel - gpt-4o-2024-11-20 ────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                ```
### High-Level Plan:

1. **Step 1 - Gather Batman Filming Locations:**  
   - Request the `web_agent` to search and compile a list of all Batman filming locations worldwide, specifying 
their cities and, if possible, precise geographic coordinates.  

2. **Step 2 - Gather Supercar Factory Locations:**  
   - Request the `web_agent` to find a list of supercar manufacturing factories worldwide, specifying their cities 
and, if possible, precise geographic coordinates.  

3. **Step 3 - Add Missing Coordinates for Locations:**  
   - For any filming locations or factories that do not have precise geographic coordinates, request the 
`web_agent` to locate and provide these using online resources like maps services or city databases.  

4. **Step 4 - Finalize Minimum Six Locations and Nearest Airports:**  
   - Combine the lists of filming locations and supercar factories to ensure there are at least six total 
locations.  
   - For each finalized location, identify the nearest major airport or cargo hub, either based on the provided 
data from the `web_agent` or by additional search if required.  

5. **Step 5 - Calculate Cargo Travel Times:**  
   - Using the exact coordinates of Gotham and the identified airports for each location, call the 
`calculate_cargo_travel_time` tool for every location to compute the transfer time via cargo plane.  

6. **Step 6 - Create a Spatial Map:**  
   - Represent the gathered locations as scatter points on a world map, color-coded based on their respective 
travel times from Gotham, using an appropriate color gradient to visualize variations in travel time.  

7. **Step 7 - Save the Map Image:**  
   - Save the visualized spatial map as an image file named **saved_map.png**.  

8. **Step 8 - Finalize the Answer:**  
   - Compile the gathered locations, their calculated travel times, and the saved map image into a coherent 
response.  
   - Use the `final_answer` tool to provide the completed solution.

\ 
                ```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
**Step 1** - Gather Batman filming locations using the `web_agent`.                                                

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
Here is your code snippet:
**Step 1** - Gather Batman filming locations using the `web_agent`.
Make sure to include code with the correct pattern, for instance:
Thoughts: Your thoughts
Code:
```py
# Your python code here
```<end_code>
Make sure to provide correct code blobs.

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The map of travel times to Gotham has been created and saved as 'saved_map.png'.")                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The map of travel times to Gotham has been created and saved as 'saved_map.png'.

Feedback:  ### Reasons Why the Plot Satisfies the Task:
1. **Correct Use of `px.scatter_geo`:**  
   The task specifies using `px.scatter_geo` for plotting, and this requirement has been met. Plotly's `scatter_geo` creates a visually appealing and interactive map, satisfying the given requirement for aesthetics.
   
2. **Representation of Batman Filming Locations and Supercar Factories:**  
   The plot contains a mix of Batman filming locations and supercar factories, ensuring the minimum requirement of six points is met. Specifically, 11 locations (5 filming and 6 factories) are represented, which exceeds the minimum threshold.

3. **Color-Coded Based on Travel Time:**  
   The travel times are represented by color coding (as shown in the gradient color bar on the right). The range of times (from 2.68 hours to 10.76 hours) is clearly visualized, enabling easy interpretation of travel differences.

4. **Titles and Annotations:**  
   - The map includes an appropriate title, "Travel Tim

[Step 6: Duration 47.67 seconds| Input tokens: 57,695 | Output tokens: 3,215]

"The map of travel times to Gotham has been created and saved as 'saved_map.png'."

In [5]:
manager_agent.python_executor.state["fig"]